In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [3]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    Trainer,
    default_data_collator
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch
from google.colab import files

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# Or wherever your files are
%cd /content/drive/MyDrive/224ndata

Mounted at /content/drive
/content/drive/MyDrive/224ndata


## Finetuning

### Dataset Preprocessing

In [ ]:
# Unresolved Error Dataset
train_data = pd.read_csv('/content/drive/MyDrive/224ndata/unres_error_train_dataset.csv')
question = "Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: "
dataset = Dataset.from_pandas(train_data)

def transform_story(example):
    story_text = example['story_truncated']
    # I think this is what it should be?
    prop = 1 - example['chunk_prop']

    formatted_text = f'[INST] Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: {story_text} [/INST] {prop} '

    return {'text': formatted_text}

transformed_dataset = dataset.map(transform_story)
for i in range(5):
    print(transformed_dataset[i]['text'])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

[INST] Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: It was a damp and dreary day in London when a telegram arrived at 221B Baker Street, summoning Sherlock Holmes and Dr. John Watson to a case that would test the limits of their deduction skills. The year was 1892, and the mystery surrounding the murder of Scott Eccles' host, Garcia, also known as the Tiger of San Pedro, was shrouded in darkness and intrigue.

With a sense of anticipation, Holmes and Watson made their way to the scene of the crime, a grand house in the outskirts of London where the gruesome murder had taken place. Miss Burnet, the governess of the household, met them with a look of fear and determination in her eyes. She revealed that she was part of a secret society seeking revenge on Garcia for his atrocities in San Pedro, a fact that sent shivers down the spines of both Holmes and Watson.

As they delved deeper into th

Finetuning Model Parameters

In [ ]:
model_name = "NeuralNovel/Mistral-7B-Instruct-v0.2-Neural-Story"

### QLoRA parameters

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

### bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

### TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# gradient_accumulation_steps = 4

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

### SFT parameters

# Maximum sequence length to use
max_seq_length = 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

Finetuning w QLoRA

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration specific to Mistral
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("neuralnovel-mistral-7b-finetuned-unresolved")

model.eval()

Your GPU supports bfloat16: accelerate training with bf16=True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


Step,Training Loss
25,1.256000
50,1.053500
75,1.032800
100,1.033300
125,0.997200
150,0.948300
175,0.986100
200,0.985000


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(
            in_features=4096, out_features=1024, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
     

### Evaluation on Test Set

In [12]:
model_name = "NeuralNovel/Mistral-7B-Instruct-v0.2-Neural-Story"
# Load base model
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


Unresolved Error

In [14]:
test_df = pd.read_csv('/content/drive/MyDrive/224ndata/unres_error_test_dataset.csv').reset_index(drop=True).dropna()
def remove_empty_lines(text):
  lines = text.split('\n')
  non_empty_lines = [line.strip() for line in lines if line.strip()]
  return '\n'.join(non_empty_lines)

# Apply the function to the text column
test_df['story'] = test_df['story'].apply(remove_empty_lines)
test_df['story_truncated'] = test_df['story_truncated'].apply(remove_empty_lines)

import re
from tqdm import tqdm
y_true = []
y_pred = []
responses = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['story_truncated']
    proportion = row['chunk_prop']
    prompt = f"[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other details. Story: {story} [/INST]"
    # Generate model output

    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs,  max_new_tokens = 50)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print(response)
    responses.append(response)

    y_true.append(proportion)

df_response = pd.DataFrame(responses, columns=['response'])
df_response.to_csv("test_neuralnovel_qlora_unresolved_responses.csv", index=False)

from google.colab import files
files.download("test_neuralnovel_qlora_unresolved_responses.csv")

  1%|          | 1/101 [00:06<11:04,  6.65s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other details. Story: In the secluded mountains behind Virginia, nestled in the mist-shrouded valleys where the trees whispered ancient secrets and the wind carried a haunting melody, there stood the stone house of Doomdorf. A man of dark repute, Doomdorf was known for his illicit liquor that held a devilish allure to those who dared to taste it. The locals whispered tales of his shady past and the chaos his hellish brew brought upon the community.
Uncle Abner, a wise and respected elder of the area, and Squire Randolph, a man of authority and influence, grew concerned about the impacts of Doomdorf's activities on the community. Together, they decided to confront the mysterious man and put an end to his nefarious deeds once and for all.
One fateful evening, they made their way to Doomdorf's stone house, determined to face him and demand an end to his brewin

  2%|▏         | 2/101 [00:11<09:27,  5.73s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other details. Story: Frances Wilton had been battling with her addiction to morphia for years, sinking deeper and deeper into its clutches until she reached her lowest point. Desperate to break free from the chains of her dependence, she sought out the help of Dr. Halifax, a renowned physician known for his unorthodox but effective methods of treating addiction.
Dr. Halifax took on Frances' case with determination and compassion, knowing the uphill battle she faced in overcoming her addiction. He devised a clever treatment plan that involved substituting morphia with strychnine, a potent tonic that mimicked some of the effects of the drug but without the addictive properties. [/INST] TheorethedraladioENDORendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorendorend

  3%|▎         | 3/101 [00:18<10:29,  6.42s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

  5%|▍         | 5/101 [00:31<10:22,  6.49s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other details. Story: In the quaint town of Willow Creek, Virginia, the first day of the circuit court brought together an unlikely trio of individuals who would soon find themselves embroiled in a captivating mystery. The pompous and well-dressed Reginald Blackwood, the rugged outdoorsman Jacob Hunter, and the tall, irascible old man Ezekiel Jenkins found themselves seated together in the courtroom, awaiting the proceedings of the day.
The case that captured their attention was the mysterious death of Duncan Moore, a well-respected elderly man in the community. His body had been discovered in the woods just outside of town, and rumors of foul play had been circulating amongst the townsfolk.
As the trial began, tensions ran high, and the three men found themselves drawn into the intricacies of the case. Reginald, with his sharp wit and keen eye for detail, 

  6%|▌         | 6/101 [00:38<10:08,  6.40s/it]

[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other details. Story: Prince Zaleski, a renowned investigator known for his sharp mind and keen intuition, was approached by a distressed nobleman seeking his help. The nobleman's family had been plagued by a series of tragic events – suicides and mysterious deaths that seemed inexplicable. Zaleski, intrigued by the challenge, agreed to take on the case.
As Zaleski delved into the investigation, he discovered a web of cryptic messages and symbols that seemed to connect the deaths to a secretive society. The symbols, etched in blood and hidden in the victims' rooms, pointed to a sinister organization with a dark agenda.
Determined to uncover the truth, Zaleski began decoding the messages and piecing together the clues. His investigations led him to a hidden chamber beneath the city, where he stumbled upon the society's headquarters. Inside, he found a group 

KeyboardInterrupt: 

Eval Metrics (subject to change)

In [ ]:
y_pred = []
responses = pd.read_csv("test_neuralnovel_qlora_unresolved_responses.csv")
for r in responses['response']:
  matches = re.findall(r'0+\.\d+', r)
  if len(matches) > 0:
    y_pred.append(1-float(matches[0]))
  else:
    y_pred.append(-1)

In [ ]:
y_pred

[0.4444444444444444,
 0.6666666666666666,
 0.34782608695652173,
 0.5454545454545454,
 0.4444444444444444,
 0.375,
 0.5,
 0.5454545454545454,
 0.375,
 0.5,
 0.6666666666666666,
 0.5,
 0.5454545454545454,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5454545454545454,
 0.5454545454545454,
 0.75,
 0.5,
 0.75,
 0.4444444444444444,
 0.5,
 0.5,
 0.4444444444444444,
 0.5,
 0.5,
 0.5454545454545454,
 0.5,
 0.5454545454545454,
 0.5454545454545454,
 0.4444444444444444,
 0.5,
 0.5,
 0.5,
 0.5,
 0.75,
 0.9444444444444444,
 0.5,
 0.5,
 0.75,
 0.5454545454545454,
 0.5,
 0.4444444444444444,
 0.5,
 0.5454545454545454,
 0.5,
 0.5,
 0.75,
 0.5,
 0.5,
 0.375,
 0.5,
 0.5,
 0.6666666666666666,
 0.5,
 0.5,
 0.5454545454545454,
 0.75,
 0.6666666666666667,
 0.5,
 0.4444444444444444,
 0.5454545454545454,
 0.5454545454545454,
 0.5,
 0.5,
 0.4444444444444444,
 0.4444444444444444,
 0.5,
 0.5454545454545454,
 0.8888888888888888,
 0.5,
 0.75,
 0.5,
 0.6666666666666666,
 0.5454545454545454,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd

test_df = pd.read_csv('/content/drive/MyDrive/224ndata/unres_error_test_dataset.csv')
test_df['pred'] = y_pred
test_df.head()
test_df.to_csv("/content/drive/MyDrive/224ndata/mistral_finetuned", index=False)

# Convert true and predicted positions to arrays for scoring
y_true = test_df["chunk_prop"]
y_pred = test_df["pred"]

# Classification Metrics
# accuracy = accuracy_score(y_true, y_pred)
# precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
# recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)
# f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

# Regression Metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r_squared = r2_score(y_true, y_pred)

# Print all metrics
# print(f'Classification Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}')
print(f'Regression Metrics:\nMSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nR-squared: {r_squared}')

Regression Metrics:
MSE: 0.019928056688087364
RMSE: 0.14116676906442027
MAE: 0.11802932211293249
R-squared: 0.35980814392765736
